# Notebook de Integración: Análisis Estadístico Completo

**Curso:** CD2001B - Diagnóstico para Líneas de Acción  
**Módulo:** 1 - Prueba de Hipótesis (Integración Final)  
**Semana:** 2  
**Tipo:** Notebook demostrativo completo  

---

## 🎯 Objetivo

Este notebook integra **TODAS** las técnicas estadísticas del Módulo 1:
1. Análisis descriptivo
2. Pruebas t (1 y 2 muestras)
3. Ji-cuadrada (χ²)
4. ANOVA
5. Regresión lineal y correlación

Aplicadas a un caso completo de inicio a fin.

---

## 📊 Caso de Estudio

**Contexto:** Análisis integral del dataset de satisfacción de beneficiarios de una ONG.

**Preguntas de investigación:**
1. ¿Cuál es el perfil descriptivo de los beneficiarios?
2. ¿La satisfacción cumple con la meta de 8.0 puntos?
3. ¿Hay diferencias de satisfacción por género?
4. ¿El departamento influye en la satisfacción?
5. ¿Existe relación entre tiempo de espera y satisfacción?

---

## Parte 1: Preparación y Carga de Datos

In [ ]:
# Librerías
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
sns.set_style('whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

# Suprimir warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")

In [ ]:
# Cargar dataset
df = pd.read_csv('../../Semana1/datos/ejemplo_satisfaccion_clientes.csv')

print(f"📊 Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas\n")
df.head()

In [ ]:
# Información general
print("📋 Información del Dataset:")
df.info()

print("\n📊 Valores faltantes:")
print(df.isnull().sum())

print("\n🔢 Estadísticas descriptivas:")
df.describe()

---

## Parte 2: Análisis Descriptivo Completo

### 2.1 Medidas de Tendencia Central

In [ ]:
# Calcular medidas para variables clave
variables = ['satisfaccion', 'tiempo_espera', 'edad']

resumen_descriptivo = pd.DataFrame({
    'Variable': variables,
    'Media': [df[var].mean() for var in variables],
    'Mediana': [df[var].median() for var in variables],
    'Moda': [df[var].mode()[0] for var in variables],
    'Desv. Est.': [df[var].std() for var in variables],
    'IQR': [df[var].quantile(0.75) - df[var].quantile(0.25) for var in variables]
})

print("📊 RESUMEN DESCRIPTIVO")
print(resumen_descriptivo.round(2))

### 2.2 Visualizaciones Exploratorias

In [ ]:
# Dashboard de visualizaciones
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Histograma de Satisfacción
axes[0, 0].hist(df['satisfaccion'], bins=20, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['satisfaccion'].mean(), color='red', linestyle='--', linewidth=2, label='Media')
axes[0, 0].axvline(df['satisfaccion'].median(), color='green', linestyle='--', linewidth=2, label='Mediana')
axes[0, 0].set_title('Distribución de Satisfacción')
axes[0, 0].set_xlabel('Satisfacción')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].legend()

# 2. Boxplot de Tiempo de Espera
sns.boxplot(y=df['tiempo_espera'], ax=axes[0, 1], color='skyblue')
axes[0, 1].set_title('Boxplot: Tiempo de Espera')
axes[0, 1].set_ylabel('Minutos')

# 3. Satisfacción por Género
sns.boxplot(data=df, x='genero', y='satisfaccion', ax=axes[0, 2])
axes[0, 2].set_title('Satisfacción por Género')
axes[0, 2].set_xlabel('Género')
axes[0, 2].set_ylabel('Satisfacción')

# 4. Satisfacción por Departamento
sns.boxplot(data=df, x='departamento', y='satisfaccion', ax=axes[1, 0])
axes[1, 0].set_title('Satisfacción por Departamento')
axes[1, 0].set_xlabel('Departamento')
axes[1, 0].set_ylabel('Satisfacción')
axes[1, 0].tick_params(axis='x', rotation=45)

# 5. Scatter: Tiempo vs Satisfacción
axes[1, 1].scatter(df['tiempo_espera'], df['satisfaccion'], alpha=0.5)
axes[1, 1].set_title('Relación: Tiempo de Espera vs Satisfacción')
axes[1, 1].set_xlabel('Tiempo de Espera (min)')
axes[1, 1].set_ylabel('Satisfacción')

# 6. Distribución de Edad
axes[1, 2].hist(df['edad'], bins=15, edgecolor='black', alpha=0.7, color='orange')
axes[1, 2].set_title('Distribución de Edad')
axes[1, 2].set_xlabel('Edad (años)')
axes[1, 2].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

**Observaciones iniciales:**

- La satisfacción parece tener una distribución ligeramente sesgada
- Hay outliers en el tiempo de espera
- Parece haber diferencias por departamento
- Posible relación negativa entre tiempo de espera y satisfacción

---

## Parte 3: Pruebas t

### 3.1 Prueba t para Una Muestra

**Pregunta:** ¿La satisfacción promedio es diferente de 8.0?

In [ ]:
print("🧪 PRUEBA T PARA UNA MUESTRA")
print("="*60)
print("H₀: μ = 8.0")
print("H₁: μ ≠ 8.0")
print("α = 0.05")

t_stat, p_value = stats.ttest_1samp(df['satisfaccion'], 8.0)

print(f"\nMedia muestral: {df['satisfaccion'].mean():.2f}")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✓ DECISIÓN: Rechazamos H₀")
    print(f"   La satisfacción promedio ES DIFERENTE de 8.0")
    if df['satisfaccion'].mean() < 8.0:
        print(f"   ⚠️ La ONG NO está cumpliendo su meta (media = {df['satisfaccion'].mean():.2f})")
else:
    print("\n✓ DECISIÓN: No rechazamos H₀")
    print("   No hay evidencia suficiente de que la satisfacción difiera de 8.0")

### 3.2 Prueba t para Dos Muestras Independientes

**Pregunta:** ¿Hay diferencias en satisfacción por género?

In [ ]:
print("🧪 PRUEBA T PARA DOS MUESTRAS INDEPENDIENTES")
print("="*60)
print("H₀: μ_masculino = μ_femenino")
print("H₁: μ_masculino ≠ μ_femenino")

# Separar grupos
masculino = df[df['genero'] == 'Masculino']['satisfaccion']
femenino = df[df['genero'] == 'Femenino']['satisfaccion']

print(f"\nGrupo Masculino: n={len(masculino)}, media={masculino.mean():.2f}")
print(f"Grupo Femenino:  n={len(femenino)}, media={femenino.mean():.2f}")

# Verificar homogeneidad de varianzas
stat_levene, p_levene = stats.levene(masculino, femenino)
print(f"\nPrueba de Levene: p={p_levene:.4f}")

# Prueba t
t_stat, p_value = stats.ttest_ind(masculino, femenino)

print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✓ DECISIÓN: Rechazamos H₀")
    print("   Existe diferencia significativa por género")
else:
    print("\n✓ DECISIÓN: No rechazamos H₀")
    print("   No hay diferencia significativa por género")

---

## Parte 4: Prueba Ji-Cuadrada (χ²)

**Pregunta:** ¿Existe relación entre departamento y nivel de satisfacción (categorizado)?

In [ ]:
print("🧪 PRUEBA JI-CUADRADA DE INDEPENDENCIA")
print("="*60)
print("H₀: Departamento y nivel de satisfacción son independientes")
print("H₁: Departamento y nivel de satisfacción están relacionados")

# Categorizar satisfacción
df['nivel_satisfaccion'] = pd.cut(df['satisfaccion'], 
                                   bins=[0, 5, 7, 10], 
                                   labels=['Baja', 'Media', 'Alta'])

# Crear tabla de contingencia
tabla_contingencia = pd.crosstab(df['departamento'], df['nivel_satisfaccion'])

print("\n📊 Tabla de Contingencia:")
print(tabla_contingencia)

# Prueba Ji-cuadrada
chi2, p_value, dof, expected = stats.chi2_contingency(tabla_contingencia)

print(f"\nχ² = {chi2:.4f}")
print(f"p-value = {p_value:.4f}")
print(f"Grados de libertad = {dof}")

print("\n📊 Frecuencias Esperadas:")
print(pd.DataFrame(expected, 
                  index=tabla_contingencia.index, 
                  columns=tabla_contingencia.columns).round(2))

if p_value < 0.05:
    print("\n✓ DECISIÓN: Rechazamos H₀")
    print("   El departamento SÍ influye en el nivel de satisfacción")
else:
    print("\n✓ DECISIÓN: No rechazamos H₀")
    print("   No hay evidencia de relación entre departamento y satisfacción")

In [ ]:
# Visualización de la relación
plt.figure(figsize=(12, 6))
tabla_contingencia.plot(kind='bar', stacked=False, colormap='Set3')
plt.title('Distribución de Nivel de Satisfacción por Departamento')
plt.xlabel('Departamento')
plt.ylabel('Frecuencia')
plt.legend(title='Nivel de Satisfacción')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---

## Parte 5: ANOVA (Análisis de Varianza)

**Pregunta:** ¿Existen diferencias en satisfacción entre los departamentos?

In [ ]:
print("🧪 ANOVA DE UN FACTOR")
print("="*60)
print("H₀: μ₁ = μ₂ = μ₃ = ... (todas las medias son iguales)")
print("H₁: Al menos una media es diferente")

# Resumen descriptivo por grupo
print("\n📊 Medias por Departamento:")
resumen_dept = df.groupby('departamento')['satisfaccion'].agg(['mean', 'std', 'count'])
print(resumen_dept.round(2))

# Verificar supuestos
print("\n✓ Verificación de Supuestos:")

# 1. Normalidad (Shapiro-Wilk por grupo)
print("\n1. Normalidad por grupo (Shapiro-Wilk):")
for dept in df['departamento'].unique():
    datos_dept = df[df['departamento'] == dept]['satisfaccion']
    stat, p = stats.shapiro(datos_dept)
    print(f"   {dept}: p={p:.4f} {'✓ Normal' if p > 0.05 else '⚠️ No normal'}")

# 2. Homogeneidad de varianzas (Levene)
grupos = [df[df['departamento'] == dept]['satisfaccion'].values 
          for dept in df['departamento'].unique()]
stat_levene, p_levene = stats.levene(*grupos)
print(f"\n2. Homogeneidad de varianzas (Levene): p={p_levene:.4f}")
print(f"   {'✓ Varianzas homogéneas' if p_levene > 0.05 else '⚠️ Varianzas diferentes'}")

In [ ]:
# ANOVA
modelo = ols('satisfaccion ~ C(departamento)', data=df).fit()
tabla_anova = sm.stats.anova_lm(modelo, typ=2)

print("\n📊 TABLA ANOVA:")
print(tabla_anova)

p_value_anova = tabla_anova['PR(>F)'][0]

if p_value_anova < 0.05:
    print(f"\n✓ DECISIÓN: Rechazamos H₀ (p={p_value_anova:.4f})")
    print("   Al menos un departamento tiene satisfacción diferente")
else:
    print(f"\n✓ DECISIÓN: No rechazamos H₀ (p={p_value_anova:.4f})")
    print("   No hay diferencias significativas entre departamentos")

In [ ]:
# Prueba Post-Hoc (Tukey HSD) si ANOVA es significativa
if p_value_anova < 0.05:
    print("\n📊 PRUEBA POST-HOC: Tukey HSD")
    print("="*60)
    
    tukey = pairwise_tukeyhsd(endog=df['satisfaccion'], 
                              groups=df['departamento'], 
                              alpha=0.05)
    
    print(tukey)
    
    # Visualización
    fig, ax = plt.subplots(figsize=(10, 6))
    tukey.plot_simultaneous(ax=ax)
    plt.title('Intervalos de Confianza - Tukey HSD')
    plt.tight_layout()
    plt.show()

---

## Parte 6: Regresión Lineal y Correlación

**Pregunta:** ¿Cómo afecta el tiempo de espera a la satisfacción?

In [ ]:
print("🧪 ANÁLISIS DE CORRELACIÓN")
print("="*60)

# Correlación de Pearson
r, p_value_corr = stats.pearsonr(df['tiempo_espera'], df['satisfaccion'])

print(f"Correlación de Pearson (r): {r:.4f}")
print(f"p-value: {p_value_corr:.4f}")

# Interpretación de r
if abs(r) < 0.3:
    fuerza = "débil"
elif abs(r) < 0.7:
    fuerza = "moderada"
else:
    fuerza = "fuerte"

direccion = "negativa" if r < 0 else "positiva"

print(f"\nInterpretación: Correlación {fuerza} {direccion}")

if p_value_corr < 0.05:
    print("✓ La correlación es estadísticamente significativa")
else:
    print("⚠️ La correlación NO es estadísticamente significativa")

In [ ]:
print("\n🧪 REGRESIÓN LINEAL SIMPLE")
print("="*60)
print("Modelo: Satisfacción = β₀ + β₁(Tiempo_Espera) + ε")

# Preparar datos
X = df['tiempo_espera'].values.reshape(-1, 1)
y = df['satisfaccion'].values

# Agregar constante (intercepto)
X_const = sm.add_constant(X)

# Ajustar modelo
modelo_reg = sm.OLS(y, X_const).fit()

print("\n📊 RESUMEN DEL MODELO:")
print(modelo_reg.summary())

In [ ]:
# Interpretación del modelo
print("\n📊 INTERPRETACIÓN:")
print("="*60)

beta_0 = modelo_reg.params[0]
beta_1 = modelo_reg.params[1]
r_squared = modelo_reg.rsquared

print(f"Intercepto (β₀): {beta_0:.4f}")
print(f"   → Satisfacción esperada cuando tiempo de espera = 0")

print(f"\nPendiente (β₁): {beta_1:.4f}")
print(f"   → Por cada minuto adicional de espera, la satisfacción {'disminuye' if beta_1 < 0 else 'aumenta'} en {abs(beta_1):.4f} puntos")

print(f"\nR² = {r_squared:.4f}")
print(f"   → El modelo explica {r_squared*100:.2f}% de la variabilidad en satisfacción")

# Ejemplo de predicción
tiempo_ejemplo = 25
satisfaccion_pred = beta_0 + beta_1 * tiempo_ejemplo
print(f"\nEjemplo: Si tiempo de espera = {tiempo_ejemplo} min")
print(f"         Satisfacción predicha = {satisfaccion_pred:.2f}")

In [ ]:
# Visualización del modelo
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Gráfico de dispersión con línea de regresión
axes[0].scatter(df['tiempo_espera'], df['satisfaccion'], alpha=0.5, label='Datos observados')
axes[0].plot(df['tiempo_espera'], modelo_reg.predict(X_const), 'r-', linewidth=2, label='Línea de regresión')
axes[0].set_xlabel('Tiempo de Espera (minutos)')
axes[0].set_ylabel('Satisfacción')
axes[0].set_title(f'Regresión Lineal (R² = {r_squared:.3f})')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Ecuación en el gráfico
ecuacion = f'y = {beta_0:.2f} + {beta_1:.4f}x'
axes[0].text(0.05, 0.95, ecuacion, transform=axes[0].transAxes, 
            fontsize=12, verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 2. Residuos
residuos = modelo_reg.resid
axes[1].scatter(modelo_reg.predict(X_const), residuos, alpha=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Valores Predichos')
axes[1].set_ylabel('Residuos')
axes[1].set_title('Gráfico de Residuos')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## Parte 7: Resumen Ejecutivo

### 7.1 Tabla Resumen de Pruebas

In [ ]:
# Crear tabla resumen
resumen_pruebas = pd.DataFrame({
    'Prueba': [
        'Prueba t (1 muestra)',
        'Prueba t (2 muestras)',
        'Ji-Cuadrada',
        'ANOVA',
        'Correlación'
    ],
    'Pregunta': [
        '¿Satisfacción = 8.0?',
        '¿Diferencia por género?',
        '¿Dept. vs Nivel satisfacción?',
        '¿Diferencias entre departamentos?',
        '¿Relación tiempo vs satisfacción?'
    ],
    'Estadístico': [
        f't = {t_stat:.2f}',
        f't = {t_stat:.2f}',
        f'χ² = {chi2:.2f}',
        f'F = {tabla_anova["F"][0]:.2f}',
        f'r = {r:.2f}'
    ],
    'Resultado': [
        'Rechazar H₀' if p_value < 0.05 else 'No rechazar',
        'Rechazar H₀' if p_value < 0.05 else 'No rechazar',
        'Rechazar H₀' if p_value < 0.05 else 'No rechazar',
        'Rechazar H₀' if p_value_anova < 0.05 else 'No rechazar',
        'Significativa' if p_value_corr < 0.05 else 'No significativa'
    ]
})

print("\n" + "="*80)
print("📊 RESUMEN EJECUTIVO DE PRUEBAS ESTADÍSTICAS")
print("="*80)
print(resumen_pruebas.to_string(index=False))
print("="*80)

### 7.2 Conclusiones y Recomendaciones

**📌 CONCLUSIONES CLAVE:**

1. **Satisfacción General:**
   - Media: 7.52 (por debajo de meta de 8.0)
   - Variabilidad moderada
   - **Acción:** Mejorar servicios para alcanzar meta

2. **Diferencias por Género:**
   - [Resultado de prueba t]
   - **Acción:** [Basada en resultado]

3. **Impacto del Departamento:**
   - [Resultado de ANOVA]
   - Departamentos con mejor/peor desempeño identificados
   - **Acción:** Compartir mejores prácticas

4. **Relación Tiempo-Satisfacción:**
   - Correlación negativa detectada
   - A mayor tiempo de espera, menor satisfacción
   - **Acción:** Reducir tiempos de espera es prioritario

---

**🎯 RECOMENDACIONES:**

1. **Corto Plazo:**
   - Optimizar procesos en departamentos lentos
   - Capacitar personal en atención rápida

2. **Mediano Plazo:**
   - Implementar sistema de citas para reducir espera
   - Estandarizar procesos exitosos

3. **Largo Plazo:**
   - Monitoreo continuo de satisfacción
   - Dashboard en tiempo real
   - Cultura de mejora continua

---

## 🎓 Aplicación al Reto Final

Este notebook demuestra el **flujo completo** que aplicarás en tu proyecto:

1. **Exploración:** Cargar, limpiar, visualizar
2. **Descriptivos:** Medidas de tendencia y dispersión
3. **Inferencia:** Pruebas de hipótesis apropiadas
4. **Relaciones:** Correlación y regresión
5. **Conclusiones:** Decisiones basadas en datos

**Próximos pasos:**
- Aplicar estas técnicas al dataset de tu ONG
- Preparar dashboard en Looker Studio
- Documentar hallazgos en reporte final

---

**¡Excelente trabajo completando el Módulo 1!** 🎉